In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import time
import tensorflow as tf
import p8_util
import p8_util_config

In [24]:
OUTPUT_DIR = './tmp/baseline'
OUTPUT_DIR_TB = './tmp'
is_tensorboard = True

#### Tensorboard processes are killed

In [25]:
# this will kill the processes for Tensorboard
#is_tensorboard = True
if is_tensorboard is True :
    !ps aux | grep tensorboard | awk '{print $2}' | xargs kill
# this will kill the processes for ngrok
if is_tensorboard is True :
    !ps aux | grep ngrok | awk '{print $2}' | xargs kill    

kill: (24160): Aucun processus de ce type
kill: (24166)

#### Increase of train dataset against test dataset.

In [26]:
OUTPUT_DIR

'./tmp/baseline'

In [27]:
import p8_util
print("Input output dir= {}".format(OUTPUT_DIR))
oNNAdaNetBuilder = p8_util.create_nn_builder(OUTPUT_DIR)
oNNAdaNetBuilder.show()

Input output dir= ./tmp/baseline

*** Number of convolutional layers= 2

*** NNAdaNetBuilder : NN Type=RNN

*** get_tf_head() : feature shape= (28, 28)

*** get_tf_head() : feature columns= [NumericColumn(key='images', shape=(28, 28), default_value=None, dtype=tf.float32, normalizer_fn=None)]

*** make_config() : output dir= ./tmp/baseline/RNN/SGRU

Max steps= 300 / Number of EPOCH=100


Global steps  : ................................... 300
NN type              : ............................ RNN
Features shape       : ............................ (28, 28)
Adanet outputdir     : ............................ ./tmp/baseline/RNN
Adanet output log    : ............................ ./tmp/baseline/RNN/SGRU
Adanet boosting iter.: ............................ 40
Adanet iter per boost: ............................ 7
Number of layers     : ............................ 2
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (

#### Customized Tensor Flow Estimator

In [28]:
oNNAdaNetBuilder = p8_util.create_nn_builder(OUTPUT_DIR)
oNNAdaNetBuilder.show()
params = {'net_builder': oNNAdaNetBuilder}


*** Number of convolutional layers= 2

*** NNAdaNetBuilder : NN Type=RNN

*** get_tf_head() : feature shape= (28, 28)

*** get_tf_head() : feature columns= [NumericColumn(key='images', shape=(28, 28), default_value=None, dtype=tf.float32, normalizer_fn=None)]

*** make_config() : output dir= ./tmp/baseline/RNN/SGRU

Max steps= 300 / Number of EPOCH=100


Global steps  : ................................... 300
NN type              : ............................ RNN
Features shape       : ............................ (28, 28)
Adanet outputdir     : ............................ ./tmp/baseline/RNN
Adanet output log    : ............................ ./tmp/baseline/RNN/SGRU
Adanet boosting iter.: ............................ 40
Adanet iter per boost: ............................ 7
Number of layers     : ............................ 2
Dropout rate         : ............................ 0.0
Seed value           : ............................ 42
Nb of classes (logit): .........................

#### Create a Baseline Estimator

In [29]:
import BaselineEstimator
oBaselineEstimator = BaselineEstimator.BaselineEstimator( params, IS_DEBUG=True)

INFO:tensorflow:Using config: {'_model_dir': './tmp/baseline/RNN/SGRU', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff63895a4e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


I0523 08:38:53.827258 140697495811904 estimator.py:201] Using config: {'_model_dir': './tmp/baseline/RNN/SGRU', '_tf_random_seed': 42, '_save_summary_steps': 10, '_save_checkpoints_steps': 10, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ff63895a4e0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [30]:
results, start_time, end_time  = oBaselineEstimator.train_and_evaluate()

INFO:tensorflow:Not using Distribute Coordinator.


I0523 08:38:53.889388 140697495811904 estimator_training.py:185] Not using Distribute Coordinator.


INFO:tensorflow:Running training and evaluation locally (non-distributed).


I0523 08:38:53.891669 140697495811904 training.py:610] Running training and evaluation locally (non-distributed).


INFO:tensorflow:Start train and evaluate loop. The evaluate will happen after every checkpoint. Checkpoint frequency is determined based on RunConfig arguments: save_checkpoints_steps 10 or save_checkpoints_secs None.


...

I0523 08:41:40.659481 140697495811904 estimator.py:1979] Saving dict for global step 300: RNN_Eval_accuracy = 0.9355, global_step = 300, loss = 0.2149805


INFO:tensorflow:Saving 'checkpoint_path' summary for global step 300: ./tmp/baseline/RNN/SGRU/model.ckpt-300


I0523 08:41:40.660974 140697495811904 estimator.py:2039] Saving 'checkpoint_path' summary for global step 300: ./tmp/baseline/RNN/SGRU/model.ckpt-300


INFO:tensorflow:Loss for final step: 0.29178038.


I0523 08:41:40.755586 140697495811904 estimator.py:359] Loss for final step: 0.29178038.


In [31]:
print("\n------------------------------------------------")
print("Time (sec)", end_time-start_time)
for key in results.keys() :
    title =key.upper()
    print("{}: {}".format(title, results[key]))
print("------------------------------------------------\n")



------------------------------------------------
Time (sec) 166.86804461479187
RNN_EVAL_ACCURACY: 0.9355000257492065
LOSS: 0.21498049795627594
GLOBAL_STEP: 300
------------------------------------------------



In [32]:
output_dir_log = './tmp/baseline/RNN/RNN'

In [33]:
oNNAdaNetBuilder.output_dir_log, is_tensorboard
output_dir_log = oNNAdaNetBuilder.output_dir_log
output_dir_log

'./tmp/baseline/RNN/SGRU'

In [34]:
if is_tensorboard is True :
    get_ipython().system_raw(
        'tensorboard --logdir {} --host localhost --port 6007 &'
        .format(output_dir_log)
    )


    get_ipython().system_raw('./assets/ngrok http 6007 &')

In [ ]:
is_tensorboard,output_dir_log